<a href="https://colab.research.google.com/github/benmsanderson/tcre-star/blob/main/get_managed_land_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get forest definition data (if not present)

In [11]:
pip install geocube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
!wget -nc https://intactforests.org/shp/IFL_2000.zip
!wget -nc https://intactforests.org/shp/Forest_zone.zip

--2023-10-01 11:13:41--  https://intactforests.org/shp/IFL_2000.zip
Resolving intactforests.org (intactforests.org)... 128.8.149.19
Connecting to intactforests.org (intactforests.org)|128.8.149.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191156796 (182M) [application/zip]
Saving to: ‘IFL_2000.zip’

IFL_2000.zip        100%[===================>] 182.30M  29.1MB/s    in 6.6s    

2023-10-01 11:13:48 (27.6 MB/s) - ‘IFL_2000.zip’ saved [191156796/191156796]

--2023-10-01 11:13:48--  https://intactforests.org/shp/Forest_zone.zip
Resolving intactforests.org (intactforests.org)... 128.8.149.19
Connecting to intactforests.org (intactforests.org)|128.8.149.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36248454 (35M) [application/zip]
Saving to: ‘Forest_zone.zip’

Forest_zone.zip     100%[===================>]  34.57M  21.9MB/s    in 1.6s    

2023-10-01 11:13:50 (21.9 MB/s) - ‘Forest_zone.zip’ saved [36248454/36248454]



Import stuff

In [14]:
import geopandas as gpd
from geocube.api.core import make_geocube
import numpy as np


In [ ]:
def get_fgrids():
  gdf = gpd.read_file('IFL_2000.zip')
  gadf = gpd.read_file('Forest_zone.zip')

  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
  gdf_crs=gdf.translate(xoff=360)
  gadf["exists"] = 1
  gdf["exists"] = 1

  out_grid=make_geocube(vector_data=gdf, measurements=["exists"], fill=np.nan, resolution=(0.05, -0.05)).fillna(0)
  out_fgrid=make_geocube(vector_data=gadf, measurements=["exists"], fill=np.nan, resolution=(0.05, -0.05)).fillna(0)

  out_grid=out_grid.rename({'x': 'lon','y': 'lat'})
  out_fgrid=out_fgrid.rename({'x': 'lon','y': 'lat'})
  return out_grid,out_fgrid


In [15]:
out_grid,out_fgrid=get_fgrids()

<ipython-input-6-4600bc800ca2>:5: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
out_grid.to_netcdf('out_grid.nc')

In [22]:
out_fgrid.to_netcdf('out_fgrid.nc')